In [6]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils
from keras.utils import to_categorical
from sklearn import metrics
import glob
import os
import librosa
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from matplotlib.pyplot import specgram
import librosa.display
from sklearn.metrics import precision_recall_fscore_support
import math

# to run this code, you'll need to load the following data:
# train_x, train_y
# valid_x, valid_y
# test_x, test_y
# see http://aqibsaeed.github.io/2016-09-24-urban-sound-classification-part-2/ for details

def windows(data, window_size):
    start = 0
    while start < len(data):
        yield start, start + window_size
        start += (window_size / 2)

def extract_features(parent_dir,sub_dirs,file_ext="*.wav",bands = 60, frames = 41):
    window_size = 512 * (frames - 1)
    log_specgrams = []
    labels = []
    for l, sub_dir in enumerate(sub_dirs):
        for fn in glob.glob(os.path.join(parent_dir, sub_dir, file_ext)):
            sound_clip,s = librosa.load(fn)
            label = fn.split('/')[2].split('-')[1]
            for (start,end) in windows(sound_clip,window_size):
                if(len(sound_clip[start:end]) == window_size):
                    signal = sound_clip[start:end]
                    melspec = librosa.feature.melspectrogram(signal, n_mels = bands)
                    logspec = librosa.logamplitude(melspec)
                    logspec = logspec.T.flatten()[:, np.newaxis].T
                    log_specgrams.append(logspec)
                    labels.append(label)

    log_specgrams = np.asarray(log_specgrams).reshape(len(log_specgrams),bands,frames,1)
    features = np.concatenate((log_specgrams, np.zeros(np.shape(log_specgrams))), axis = 3)
    for i in range(len(features)):
        features[i, :, :, 1] = librosa.feature.delta(features[i, :, :, 0])

    return np.array(features), np.array(labels,dtype = np.int)

def one_hot_encode(labels):
    n_labels = len(labels)
    n_unique_labels = len(np.unique(labels))
    one_hot_encode = np.zeros((n_labels,n_unique_labels))
    one_hot_encode[np.arange(n_labels), labels] = 1
    return one_hot_encode

parent_dir = 'UrbanSound'
tr_sub_dirs = ["fold1","fold2","fold3","fold4"]
tr_features,tr_labels = extract_features(parent_dir,tr_sub_dirs)
#tr_labels = one_hot_encode(tr_labels)
tr_labels = to_categorical(tr_labels,num_classes=10)
train_x = tr_features
train_y = tr_labels


vl_sub_dirs = ["fold5"]
vl_features,vl_labels = extract_features(parent_dir,vl_sub_dirs)
#vl_labels = one_hot_encode(vl_labels)
vl_labels = to_categorical(vl_labels,num_classes=10)
val_x = vl_features
val_y = vl_labels

ts_sub_dirs= ['fold10test']
ts_features,ts_labels = extract_features(parent_dir,ts_sub_dirs)
#ts_labels = one_hot_encode(ts_labels)
ts_labels = to_categorical(ts_labels,num_classes=10)
test_x = ts_features
test_y = ts_labels

# data dimension parameters
frames = 41
bands = 60
num_channels = 2
num_labels = test_y.shape[1]

# start by creating a linear stack of layers
model = Sequential()

# will use filters of size 2x2
f_size = 2

# first layer applies 32 convolution filters
# input: 60x41 data frames with 2 channels => (60,41,2) tensors
model.add(Convolution2D(32, f_size, f_size, border_mode='same', input_shape=(bands, frames, num_channels)))
model.add(Activation('relu'))
model.add(Convolution2D(32, f_size, f_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.15))

# next layer applies 64 convolution filters
model.add(Convolution2D(64, f_size, f_size, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, f_size, f_size))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

# flatten output into a single dimension
# Keras will do the shape inference automatically
model.add(Flatten())

# then a fully connected NN layer
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))

# finally, an output layer with one node per class
model.add(Dense(num_labels))
model.add(Activation('softmax'))

# use the Adam optimiser
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

# now compile the model, Keras will take care of the Tensorflow boilerplate
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer=adam)

print(train_x.shape)
print(train_y.shape)
print(type(train_x.shape))
print(type(train_y.shape))
print(train_x)
print(train_y)
# for quicker training, just using one epoch, you can experiment with more
model.fit(train_x, train_y, validation_data=(val_x, val_y), batch_size=32, nb_epoch=8)

# finally, evaluate the model using the withheld test dataset

/home/paperspace/Desktop/NoisePollutionDetection/NoisePollution/lib/python2.7/site-packages/ipykernel_launcher.py:101: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2), padding="same", input_shape=(60, 41, 2...)`
/home/paperspace/Desktop/NoisePollutionDetection/NoisePollution/lib/python2.7/site-packages/ipykernel_launcher.py:103: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (2, 2))`
/home/paperspace/Desktop/NoisePollutionDetection/NoisePollution/lib/python2.7/site-packages/ipykernel_launcher.py:109: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2), padding="same")`
/home/paperspace/Desktop/NoisePollutionDetection/NoisePollution/lib/python2.7/site-packages/ipykernel_launcher.py:111: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (2, 2))`


(22734, 60, 41, 2)
(22734, 10)
<type 'tuple'>
<type 'tuple'>
[[[[  1.23031468e+01  -1.87125731e+00]
   [  1.29379819e+01  -3.37924419e+00]
   [  1.06374181e+01  -4.02307577e+00]
   ..., 
   [ -4.45965335e+01  -3.04084162e-01]
   [ -4.40234278e+01  -1.84795853e-01]
   [ -4.46492471e+01  -3.99762901e-01]]

  [[ -4.34858214e+01  -4.89159477e-01]
   [ -4.76328760e+01  -2.99652177e-01]
   [ -4.40518867e+01  -2.02767992e-01]
   ..., 
   [ -3.25658220e+01  -2.77502390e-01]
   [ -3.23546238e+01  -1.37646609e-01]
   [ -3.48327878e+01  -1.73643064e-02]]

  [[ -3.91067720e+01   5.60110219e-02]
   [ -3.85909486e+01   1.31396345e-01]
   [ -3.97081374e+01   9.99451434e-02]
   ..., 
   [  1.07936528e+01   1.05278158e+01]
   [  1.05684197e+01   9.49338966e+00]
   [  1.06907560e+01   7.42744991e+00]]

  ..., 
  [[ -5.61721435e+01   7.55582673e+00]
   [ -5.68719109e+01   9.87286893e+00]
   [ -6.19945198e+01   1.11977640e+01]
   ..., 
   [ -4.44076655e+01  -6.34758013e-01]
   [ -4.38905881e+01  -5.256523

Train on 22734 samples, validate on 5689 samples
Epoch 1/8
22734/22734 [==============================] - 113s 5ms/step - loss: 1.2965 - acc: 0.5864 - val_loss: 2.4064 - val_acc: 0.3959
Epoch 2/8
22734/22734 [==============================] - 115s 5ms/step - loss: 0.5590 - acc: 0.8109 - val_loss: 2.5332 - val_acc: 0.4134
Epoch 3/8
22734/22734 [==============================] - 113s 5ms/step - loss: 0.3859 - acc: 0.8692 - val_loss: 3.1825 - val_acc: 0.4384
Epoch 4/8
22734/22734 [==============================] - 114s 5ms/step - loss: 0.2988 - acc: 0.8946 - val_loss: 3.6050 - val_acc: 0.3967
Epoch 5/8
22734/22734 [==============================] - 114s 5ms/step - loss: 0.2406 - acc: 0.9189 - val_loss: 5.0531 - val_acc: 0.3739
Epoch 6/8
22734/22734 [==============================] - 114s 5ms/step - loss: 0.2106 - acc: 0.9270 - val_loss: 4.8136 - val_acc: 0.3897
Epoch 7/8
22734/22734 [==============================] - 115s 5ms/step - loss: 0.1911 - acc: 0.9351 - val_loss: 4.5787 - val_acc:

In [7]:
# determine the ROC AUC score
y_prob = model.predict_proba(test_x, verbose=0)
print(len(y_prob))
#y_pred = np_utils.probas_to_classes(y_prob)
##y_true = np.argmax(test_y, 1)
#roc = metrics.roc_auc_score(test_y, y_prob)
#print "ROC:", round(roc,3)

# determine the classification accuracy
score, accuracy = model.evaluate(test_x, test_y, batch_size=32)
print("\nAccuracy = {:.2f}".format(accuracy))


838
838/838 [==============================] - 1s 1ms/step

Accuracy = 0.72


In [8]:
# determine the ROC AUC score
import h5py

In [10]:
model.save('my_model.h5')